In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        # 6 input image channel, 16 output channels, 3x3 square convolution kernel        
        self.conv2 = nn.Conv2d(6, 16, 3)

        # an affine operations: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120) # 6*6 from image dimention
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Whenever creating your own nn.Module implementation, 
        # you have to define the `forward` method.
        # The `backward` funciton (for gradient calculation) 
        # is automatically defined via autograd.
        
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
        

In [13]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [14]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight size

10
torch.Size([6, 1, 3, 3])


In [20]:
input = torch.randn(1, 1, 32, 32) # random 32x32 input
output = net(input)
print(output)

tensor([[ 0.0585,  0.0595,  0.0349,  0.0730,  0.0744, -0.0132, -0.0668, -0.0632,
         -0.1290, -0.0956]], grad_fn=<AddmmBackward>)


In [21]:
target = torch.randn(10)
target = target.view(1, -1) # reshape the target tensor to the same shape as output
criterion = nn.MSELoss() #Severeal loss functions are available in nn package
loss = criterion(output, target)
print(loss)
# net.zero_grad()
# out.backward(torch.randn(1, 10)) # backpropagate the errors to calculate the gradients

tensor(1.0119, grad_fn=<MseLossBackward>)


In [22]:
# the graph computaion now should look like this:
# input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d 
#       -> view -> linear -> relu -> linear -> relu -> linear 
#       -> MSELoss
#       -> loss

# Thus, by calling loss.backward(), the whole graph is differentiated w.r.t. the loss value,
# and all the tensors in the graph that has requires_grad=True will have their .grad tensor
# accumulated with the gradient. Magic!

print(loss.grad_fn) #MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLu


# Backprop the gradients
We can propagate the error by using `loss.backward()` method. But first, you need to clear the existing gradients. Otherwise, gradients will be accumulated.

In [23]:
net.zero_grad()

print(f"net.conv1.bias.grad before backward: {net.conv1.bias.grad}")

loss.backward()
print(f"net.conv1.bias.grad after backward: {net.conv1.bias.grad}")

net.conv1.bias.grad before backward: tensor([0., 0., 0., 0., 0., 0.])
net.conv1.bias.grad after backward: tensor([-0.0121,  0.0167, -0.0187,  0.0097, -0.0055, -0.0093])


# Update the net weights

Usually the weights are updated using a very simple rule: `weight = weight - learning_rate * gradient`.

```python
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
```
However, in a neural network you want to use something better. Various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, are implemented in the package `torch.optim`.

In [25]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.01)     # Using Stochastic Gradient Descent to update the weights

optimizer.zero_grad()                                # clean up the gradient buffers, otherwise they will accumulate
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()                                     # Perform one step of the SGC optimization,
                                                     # updating the network weights and biases.
                                                     # Use this inside the training loop
        